## Guided Project: Answering Business questions using SQL

In this guided project we will use the Chinook database and answer business questions using SQL.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### List of tables and views in the database

In [2]:
%%sql
SELECT name,
       type
FROM sqlite_master
WHERE type IN ('table', 'view')
       

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Explore the data tables

In [3]:
%%sql
SELECT *
FROM album
LIMIT 5

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [4]:
%%sql
SELECT *
FROM artist
LIMIT 5

Done.


artist_id,name
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


In [5]:
%%sql
SELECT *
FROM customer
LIMIT 5

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


### 1. Which genre sell most in USA

In [6]:
%%sql
WITH usa_tracks_sold AS 
    (SELECT * 
     FROM invoice_line il
          INNER JOIN invoice i ON i.invoice_id = il.invoice_id
          INNER JOIN customer c ON c.customer_id = i.customer_id
     WHERE c.country == 'USA'
           )

SELECT g.name genre,
       COUNT(uts.invoice_line_id) tracks_sold,
       CAST(COUNT(uts.invoice_line_id) AS float)/(SELECT COUNT(*)
                                                 FROM usa_tracks_sold) AS percentage_sold
    
       
FROM usa_tracks_sold uts
INNER JOIN track t ON t.track_id = uts.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


We are given 4 artists names by the genre from USA to recommend three artists albums to purchase for the store based on popularity. For this we need to analyze which genre sells best in the USA. The given 4 artists names and genre are as below.
   * Regal : Hip-Hop
   * Red Tone : Punk
   * Meteor and the Girls : Pop
   * Slim Jim Bites : Blues

From the results genre 'Punk' does not have any tracks selling currently. The highest selling genres are Blues, Pop and Hip Hop. Therefore based on the sales tracks only the artists Slim Jim Bites, Meteor and the Girls, Regal albums should be purchased for the store.

### 2. Performance of Sales support agents

In [7]:
%%sql
WITH customer_support AS
     (SELECT *
      FROM employee e
      INNER JOIN customer c ON c.support_rep_id = e.employee_id
      INNER JOIN invoice i ON i.customer_id = c.customer_id)

SELECT cs.employee_id,
       cs.first_name || ' ' || cs.last_name AS employee_name,
       SUM(cs.total) AS total_sales
FROM customer_support cs
GROUP BY 1
ORDER BY 3 DESC

Done.


employee_id,employee_name,total_sales
3,Jane Peacock,1731.5100000000039
4,Margaret Park,1584.0000000000034
5,Steve Johnson,1393.920000000002


The above analysis was find the performance of the sales support agent. Each customer is assigned to a sales support agent within the company for their first purchase. The above query has calculate the total sales that the sales support agent make.

From the resutls there is not a big difference between the 3 sales agents total sales amount for the customers. Jane Peacock agent has made the highest sales of 1731.51 dollars. Second and third highest is Margaret Park and Steve Johnson.  

### 3. Analyze sales data for customers from each country

In [8]:
%%sql
WITH other AS 
    (SELECT COUNT(DISTINCT(c.customer_id)) AS cus_count,
            c.country,
            SUM(il.unit_price) sales,
            COUNT(DISTINCT(i.invoice_id)) AS orders,
            CASE 
                WHEN (SELECT COUNT(DISTINCT(c.customer_id))
                      FROM customer
                      GROUP BY country)
                      >1 THEN c.country
            ELSE 'Other'
            END AS country_2
     FROM customer c
     INNER JOIN invoice i ON i.customer_id = c.customer_id
     INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
     GROUP BY country
    ),
    
summary AS (SELECT  o.country_2 AS country,
                    SUM(o.cus_count) AS total_customers,
                    SUM(o.sales) AS total_sales,
                    SUM(o.sales)/SUM(o.cus_count) AS avg_order_per_cus,
                    SUM(o.sales)/SUM(o.orders) AS average_order_value
            FROM other o
            GROUP BY 1)

SELECT  country,
        total_customers,
        total_sales,
        avg_order_per_cus,
        average_order_value
FROM (
        SELECT *,
             CASE 
                 WHEN s.country <>'Other' THEN 1
                 ELSE 0
                 END AS sort
        FROM summary s
        ORDER BY total_customers DESC)
ORDER BY sort DESC

Done.


country,total_customers,total_sales,avg_order_per_cus,average_order_value
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
India,2,183.1500000000002,91.5750000000001,8.72142857142858
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
Other,15,1094.9399999999994,72.99599999999995,7.448571428571424


The analysis was to find the below questions from each country.
   1. total number of customers
   2. total value of sales
   3. average value of sales per customer
   4. average order value

The number of customers with one for each country was grouped as 'Other' and forced the ordering of 'Other' to the last in our analysis in the main query.

From the results we can see that USA has the highest number of customers 13, then Canada, Brazil and France. There is 15 countries with only one customer.

### 4. Analyze album purchases

In [9]:
%%sql
WITH invoice_info AS
    (SELECT il.invoice_id invoice_id,
            t.album_id album_id,
            t.track_id track_id
     FROM invoice_line il 
     INNER JOIN track t ON t.track_id = il.track_id
     GROUP BY 1)
    
SELECT album_purchase,
        COUNT(DISTINCT(invoice_id)) AS num_invoices,
        CAST(COUNT(DISTINCT(invoice_id)) AS FLOAT)
        /(SELECT COUNT(DISTINCT(inv.invoice_id))
                            FROM invoice_info inv ) AS percentage
        
FROM
    (SELECT *,
           CASE 
            WHEN( (SELECT track_id FROM track t
                   WHERE t.album_id = inv.album_id
                   EXCEPT 
                   SELECT track_id FROM invoice_line il
                   WHERE il.invoice_id = inv.invoice_id) IS NULL           
                 
                   AND
                 
                   (SELECT track_id FROM invoice_line il
                   WHERE il.invoice_id = inv.invoice_id
                   EXCEPT 
                   SELECT track_id FROM track t
                   WHERE t.album_id = inv.album_id)) IS NULL
            THEN 'Yes'
            ELSE 'No'
            END AS album_purchase
    FROM invoice_info inv)
GROUP BY 1

Done.


album_purchase,num_invoices,percentage
No,500,0.8143322475570033
Yes,114,0.18566775244299674


Chinook store is setup in a way that allows customers to make purchases as a whole album and collection of one or more individual tracks. For this question we have to identify the below.
          1. album purchase or individual purchase.
          2. For each purchase the number of invoices.
          3. For each purchase the invoice percentage.
          
To anwer these question first we have to identify whether each invoice has all the tracks from an album. For this we have to get a list of tracks from an invoice and compare it to list of tracks from an album. To compare this we have used EXCEPT operator.

From the results we can see there is 114 album purchases and 500 track purchases.

### 5. Most popular artist

In [10]:
%%sql 
SELECT  a.name AS artist_name,
        COUNT(DISTINCT(p.playlist_id)) AS playlist_count
FROM artist a
INNER JOIN album al ON al.artist_id = a.artist_id
INNER JOIN track t ON t.album_id = al.album_id
INNER JOIN playlist_track pt ON pt.track_id = t.track_id
INNER JOIN playlist p ON p.playlist_id = pt.playlist_id
GROUP BY 1
ORDER BY 2 DESC

Done.


artist_name,playlist_count
Eugene Ormandy,7
Academy of St. Martin in the Fields & Sir Neville Marriner,6
Berliner Philharmoniker & Herbert Von Karajan,6
English Concert & Trevor Pinnock,6
The King's Singers,6
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",5
Adrian Leaper & Doreen de Feis,5
Alberto Turco & Nova Schola Gregoriana,5
Antal Doráti & London Symphony Orchestra,5
Barry Wordsworth & BBC Concert Orchestra,5


For the most popular artist is Eugene Ormandy highest with 7 playlist_counts and second highest there are few artist with 6 playlist_counts, Academy of St. Martin in the Fields & Sir Neville Marriner, Berliner Philharmoniker & Herbert Von Karajan, English Concert & Trevor Pinnock and The King's Singers.

### 6. Tracks puchased vs not purchased

In [11]:
%%sql
WITH all_sold_tracks AS
(SELECT COUNT(DISTINCT(t.track_id)) AS all_tracks,
        COUNT(DISTINCT(il.track_id)) sold_tracks
 FROM track t
 LEFT JOIN invoice_line il ON t.track_id = il.track_id
 
)

SELECT all_tracks,
       sold_tracks,
       (CAST(sold_tracks AS float)/all_tracks) * 100 percentage_sold
    FROM all_sold_tracks

Done.


all_tracks,sold_tracks,percentage_sold
3503,1806,51.55580930630887


From the above query there are 3503 of total tracks and out of that 1806 are being sold. The percentage of tracks sold as 51.56%

### 7. Is the range of tracks in store reflective of their sales popularity

In [12]:
%%sql
WITH summary AS
(SELECT g.name AS genre_name,
        COUNT(DISTINCT(t.track_id)) AS track_count,
        COUNT(DISTINCT(il.track_id)) AS tracks_sold
 FROM genre g
 INNER JOIN track t ON t.genre_id = g.genre_id
 LEFT JOIN invoice_line il ON il.track_id = t.track_id
 GROUP BY 1
 ORDER BY 3 DESC
)

SELECT *
FROM summary

Done.


genre_name,track_count,tracks_sold
Rock,1297,915
Metal,374,238
Alternative & Punk,332,176
Latin,579,119
Jazz,130,61
Blues,81,56
R&B/Soul,61,55
Alternative,40,34
Electronica/Dance,30,29
Pop,48,25


From Rock genre there is highest number of tracks available and the highest number of tracks are being sold. There is below 50% of tracks being sold from Latin genre. From Drama, TV shows and Sound track there is 50 to 100 tracks available less than 10 tracks have being sold. We can conclude that the range of tracks in store is some what reflective on the sales popularity.

### 8. Do protected vs non-protected media types have an effect on popularity?

In [13]:
%%sql
SELECT*
FROM media_type

Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


In [14]:
%%sql
WITH summary AS
(SELECT  COUNT(DISTINCT(t.track_id)) AS all_tracks,
         COUNT(DISTINCT(il.track_id)) AS tracks_sold,
         CASE WHEN (m.name) LIKE '%Protected%'
              THEN 'protected'
              ELSE 'Non-protected'
              END AS Media_type
 FROM media_type m
 INNER JOIN track t ON t.media_type_id = m.media_type_id
 LEFT JOIN invoice_line il ON il.track_id = t.track_id
 GROUP BY Media_type
)

SELECT  Media_type,
        all_tracks,
        ROUND(CAST(all_tracks AS FLOAT)/(SELECT SUM(all_tracks) FROM summary),4)*100 AS all_track_perc, 
        tracks_sold,
        ROUND(CAST(tracks_sold AS FLOAT)/(SELECT SUM(tracks_sold) FROM summary),4)*100 AS tracks_sold_perc 
       
    FROM summary

Done.


Media_type,all_tracks,all_track_perc,tracks_sold,tracks_sold_perc
Non-protected,3052,87.13,1652,91.47
protected,451,12.870000000000001,154,8.53


From the above query we can conclude that there are more media non-protected tracks and very few media-protected tracks. from that very high percentage of non-protected tracks are being sold making it popular.

### Conclusion

1. The highest tracks are sold from 'Rock' genre with 53.4% sales. Then second and third highest from 'Alternative & Punk' and 'Metal'
2. Jane Peacock is the best sales performance agent with sales amount of 1731.51. There is not a big difference between the top 3 agents. The second and thrid best sales performance are from Margaret Park and Steve Johnson.
3. Highest number of customers are from USA then Canda and Brazil. There are 15 countries with one customer from each country.
4. There is 81.4% of customers purchasing individual tracks and 18.6% of album purchases.
5. The most popular artist is Eugene Ormandy.
6. The percentage of tracks being sold is 51.56% from total of tracks.
7. The range of tracks in store is some what relective of the sales popularity.
8. From 87.13% total non-protected tracks, 91.47% of tracks sold was non-protected.